In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("/content/train.csv",error_bad_lines=False, engine="python")
df.head(3)


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 196: unexpected end of data


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1


In [ ]:
df.isnull().shape

(194, 5)

In [ ]:
import nltk
from nltk.corpus import stopwords
import re 

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
a=df["title"]
for i in a:
  print(i)

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
df.dropna(how="any",inplace=True)

In [ ]:
messages=df["title"]
print((messages))

0      House Dem Aide: We Didn’t Even See Comey’s Let...
1      FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                      Why the Truth Might Get You Fired
3      15 Civilians Killed In Single US Airstrike Hav...
4      Iranian woman jailed for fictional unpublished...
                             ...                        
189    UK citizens and war heroes get cheap pre-fab h...
190    After Vets Fight War, Feds Demand Money Back… ...
191           Mr. Trump’s Wild Ride - The New York Times
192    Here Is How FBI Director Comey BAMBOOZLED The ...
193    20 Foods That Naturally Unclog Arteries and Pr...
Name: title, Length: 170, dtype: object


In [ ]:
# data Preprocessing such as remove stopwords,lowering and apply stemming 
def data_preprocessing(input_messages):
  data=[]
  for i in input_messages :
    
    review=re.sub('[^a-zA-Z]'," ",i)
    review=review.lower()
    review=review.split()
    
    review=[ps.stem(word) for word in review if word not in stopwords.words("english")]
    review=" ".join(review)
    data.append(review)
  return data


training_data=data_preprocessing(messages)


['hous dem aid even see comey letter jason chaffetz tweet', 'flynn hillari clinton big woman campu breitbart', 'truth might get fire', 'civilian kill singl us airstrik identifi', 'iranian woman jail fiction unpublish stori woman stone death adulteri', 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart', 'beno hamon win french socialist parti presidenti nomin new york time', 'back channel plan ukrain russia courtesi trump associ new york time', 'obama organ action partner soro link indivis disrupt trump agenda', 'bbc comedi sketch real housew isi caus outrag', 'russian research discov secret nazi militari base treasur hunter arctic photo', 'us offici see link trump russia', 'ye paid govern troll social media blog forum websit', 'major leagu soccer argentin find home success new york time', 'well fargo chief abruptli step new york time', 'anonym donor pay million releas everyon arrest dakota access pipelin', 'fbi close hillari', 'chuck todd

In [ ]:
# wwe are going to implement the whole code using tensorflow
!pip install tensorflow


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# importing different required libaries 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout

In [ ]:
# we are specifing the vocab size (unique words)
vc_size=300

In [ ]:
import numpy as np


In [ ]:
# applying the one-hot encoding method.
# where vc_size can be used as tuning parameter


# it is very important to have input with same diamesion, using max_len 40 we are converting each sentence into 40 different features, padding can be pre of post
# [135, 184, 266, 88, 114, 85, 297, 134, 44, 129]  after pre padding this will beacuse [0,0,0,0,0,0,0,0,135, 184, 266, 88, 114, 85, 297, 134, 44, 129] and 
# [135, 184, 266, 88, 114, 85, 297, 134, 44, 129,0,0,0,0,0,0,0,0,0] called post padding where max_len is 18. this for example to show what is pre and post padding
def word_conversion(vocbulary_size,data_for_traing):
  one_hot_format=[one_hot(doc,vc_size) for doc in data_for_traing]
  word_embedding=pad_sequences(one_hot_format,maxlen=40,padding="pre")
  return word_embedding
encode_word=word_conversion(vc_size,training_data)
print(encode_word.shape)


(170, 40)


In [ ]:
input_diamension=40 # as we have featurized each word in 40 diamensions

In [ ]:
# let's build the model for classification of fake new using LSTM
# first layer must be embedding layer
# second: LSTM 
# then Dense layer along with activation function, sigmoid mainly for binary classification

#addition touch duitn training
# we can also try to chekc purformance of model using dropout layer at different positions
output_size=30
sent_lenght=40
model=Sequential()
model.add(Embedding(sent_lenght,output_size))

model.add(LSTM(100))

model.add(Dense(1,activation="sigmoid"))
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=["accuracy"])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, None, 30)          1200      
                                                                 
 lstm_4 (LSTM)               (None, 100)               52400     
                                                                 
 dense_4 (Dense)             (None, 1)                 101       
                                                                 
Total params: 53,701
Trainable params: 53,701
Non-trainable params: 0
_________________________________________________________________


In [ ]:
df.dropna()
# prepare x and y data, civert them into array form
y=np.array(df["label"])
x=np.array(encode_word)

print(x.shape,y.shape)



(170, 40) (170,)


In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
# splitting the data into train and test uisng sklearn method
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.2)


In [ ]:
# training the model 
model.fit(train_x,train_y,validation_data=(test_x,test_y),epochs=15,shuffle=False,batch_size=50)

Epoch 1/15
3/3 [==============================] - 2s 228ms/step - loss: 0.6922 - accuracy: 0.5221 - val_loss: 0.6892 - val_accuracy: 0.5882
Epoch 2/15
3/3 [==============================] - 0s 17ms/step - loss: 0.6880 - accuracy: 0.5956 - val_loss: 0.6849 - val_accuracy: 0.5882
Epoch 3/15
3/3 [==============================] - 0s 17ms/step - loss: 0.6830 - accuracy: 0.5956 - val_loss: 0.6793 - val_accuracy: 0.5882
Epoch 4/15
3/3 [==============================] - 0s 16ms/step - loss: 0.6766 - accuracy: 0.5956 - val_loss: 0.6737 - val_accuracy: 0.5882
Epoch 5/15
3/3 [==============================] - 0s 16ms/step - loss: 0.6707 - accuracy: 0.5956 - val_loss: 0.6719 - val_accuracy: 0.5882
Epoch 6/15
3/3 [==============================] - 0s 16ms/step - loss: 0.6653 - accuracy: 0.5956 - val_loss: 0.6696 - val_accuracy: 0.5882
Epoch 7/15
3/3 [==============================] - 0s 25ms/step - loss: 0.6596 - accuracy: 0.5956 - val_loss: 0.6704 - val_accuracy: 0.6176
Epoch 8/15
3/3 [==========

In [ ]:
from sklearn.metrics import accuracy_score,classification_report

In [ ]:
y_pred=model.predict(test_x)

2/2 [==============================] - 0s 7ms/step


In [ ]:
y_pred=np.where(y_pred > 0.5,1,0)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
accuracy_score(test_y,y_pred)

0.7535483870967742

In [ ]:
classification_report(test_y,y_pred)

'              precision    recall  f1-score   support\n\n           0       0.76      0.83      0.80       452\n           1       0.73      0.64      0.68       323\n\n    accuracy                           0.75       775\n   macro avg       0.75      0.74      0.74       775\nweighted avg       0.75      0.75      0.75       775\n'

In [ ]:
def model_testing(input_sentence):
  test_sen=[]
  if 
    review=re.sub('[^a-zA-Z]'," ",input_sentence)
    review=review.lower()
    review=review.split()
    
    review=[ps.stem(word) for word in review if word not in stopwords.words("english")]
    review=" ".join(review)
    test_sen.append(review)
  return test_sen

